In [1]:
import os
import pickle

from langchain.chains import GraphCypherQAChain
from langchain_community.document_loaders import TextLoader
from langchain_community.graphs import Neo4jGraph
from langchain_experimental.graph_transformers.llm import LLMGraphTransformer
from langchain_openai import AzureChatOpenAI, ChatOpenAI
from langchain_text_splitters.character import RecursiveCharacterTextSplitter

In [2]:
# Deepseek
# API_KEY = os.getenv("DEEPSEEK_API_KEY")
# ENDPOINT = "https://api.deepseek.com"
# MODEL_NAME = "deepseek-chat"

# Github marketplace
# API_KEY = os.getenv("GITHUB_MARKETPLACE_API_KEY")
# ENDPOINT = "https://models.inference.ai.azure.com"
# MODEL_NAME = "gpt-4o"

# Cloudflare Worker AI
API_KEY = os.getenv("CLOUDFLARE_WORKER_AI_API_KEY")
ACCOUNT_ID = os.getenv("CLOUDFLARE_WORKER_AI_ACCOUNT_ID")
ENDPOINT = f"https://api.cloudflare.com/client/v4/accounts/{ACCOUNT_ID}/ai/v1"
MODEL_NAME = "@cf/meta/llama-3.1-8b-instruct"

os.environ["SSL_CERT_FILE"] = os.getenv("REQUESTS_CA_BUNDLE")

llm = ChatOpenAI(
    base_url=ENDPOINT,
    model=MODEL_NAME,
    api_key=API_KEY,
    temperature=0,
)

In [9]:
# Load the document
loaded_documents = TextLoader(file_path="imdbtop3.txt").load()

# Split the document into chunks
documents_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=300)
splitted_documents = documents_splitter.split_documents(documents=loaded_documents)

In [10]:
# Transform documents into graph-based documents using a LLM
# Set allowed nodes and relationships
llm_graph_transformer = LLMGraphTransformer(
    llm=llm,
)

# Convert the splitted documents into graph documents
graph_documents = llm_graph_transformer.convert_to_graph_documents(splitted_documents)

In [11]:
# NEO4J connection
neo4j_uri = "bolt://localhost:7183"
neo4j_user = "neo4j"
neo4j_password = "langchain"

graph = Neo4jGraph(url=neo4j_uri, username=neo4j_user, password=neo4j_password)
graph.add_graph_documents(graph_documents, baseEntityLabel=False, include_source=False)

# MATCH (n) DETACH DELETE n;
# MATCH (n)-[r]->(m) RETURN n, r, m;

In [ ]:
# Visualize the graph
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget

# NEO4J connection
neo4j_uri = "bolt://localhost:7183"
neo4j_user = "neo4j"
neo4j_password = "langchain"

def showGraph():
    driver = GraphDatabase.driver(uri = neo4j_uri, auth = (neo4j_user, neo4j_password))
    session = driver.session()
    widget = GraphWidget(graph = session.run("MATCH (n)-[r]->(m) RETURN n, r, m;").graph())
    widget.node_label_mapping = 'id'
    return widget

showGraph()

In [ ]:
# Query NEO4J with natural language

graph.refresh_schema()
#  print(graph.schema)

neo4j_query = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True,
)

message = neo4j_query.invoke({"query": "Who acted in The Shawshank Redemption, and the according role"})
print(message)